In [132]:
!pip3 install -q -U google-generativeai

In [1]:
import time
from tqdm import tqdm
import json
from dotenv import load_dotenv
import os
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
import google.generativeai as genai
from IPython.display import Image, display
import PIL.Image
import pandas as pd

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

/opt/miniconda3/envs/ml_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set model parameters
generation_config = genai.GenerationConfig(
    temperature=0.8,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

# Set safety settings
safety_settings = [
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
]

In [4]:
df = pd.read_csv("csv_files/200-annotated.csv")

In [5]:
columns_to_keep = ["submissionId", "submissionTitle", "isTextSarcastic?", "isImageSarcastic?", "isTogetherSarcastic?"]
df = df[columns_to_keep]

In [6]:
df.head()

,submissionId,submissionTitle,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?
0,8q9bjm,finally someone said it\n he's right. you are ...,0,1,1
1,glf93t,pure evil\n this attempt on my life has left m...,0,1,1
2,kctb6e,sounded way better in my head\n i remember whe...,0,1,1
3,j9qaro,totally relatable\n why do we do this?! i dont...,0,1,1
4,i5z2pf,we weren’t expecting special forces\n this guy...,0,1,1


### Initializing new columns for llm generated annotations

In [7]:
df['i_s'] = None
df.head()

,submissionId,submissionTitle,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,i_s
0,8q9bjm,finally someone said it\n he's right. you are ...,0,1,1,None
1,glf93t,pure evil\n this attempt on my life has left m...,0,1,1,None
2,kctb6e,sounded way better in my head\n i remember whe...,0,1,1,None
3,j9qaro,totally relatable\n why do we do this?! i dont...,0,1,1,None
4,i5z2pf,we weren’t expecting special forces\n this guy...,0,1,1,None


In [8]:
def provide_prompt_for_image_labels():
    image_prompt = f"""
    TASK:
    You will be given an image, and your task is to determine whether the image contains sarcasm. 
    Indicate your response with a 1 if sarcasm is detected and a 0 if it is not sarcastic.

    EXAMPLES:
    """
    examples = [
        ("Sample Images/ayezy8.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/epymwl.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/tumor.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/oil_painting.png", '{"isImageSarcastic?": 1}'),
        ("Sample Images/cable.png", '{"isImageSarcastic?": 0}'),
        ("Sample Images/snake.png", '{"isImageSarcastic?": 0}'),
        ("Sample Images/Traffic_bounded.png", '{"isImageSarcastic?": 0}'),
        ("Sample Images/university.png", '{"isImageSarcastic?": 0}'),
    ]

    message = [image_prompt]
    for image, resp in examples:
        message = message + [
            PIL.Image.open(image),
            resp
        ]

    return message

# Defining a response schema in order to obtain consistently formatted response
response_schema_image = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "isImageSarcastic?": {
                "type": "integer",
            },
        },
        "required": ["isImageSarcastic?"],
    },
}

def detect_sarcasm_in_image(image:str) -> str:
    response = model.generate_content(
        provide_prompt_for_image_labels() + [
            PIL.Image.open(image)
        ],
        safety_settings=safety_settings,
        generation_config = genai.GenerationConfig(
            response_mime_type="application/json", response_schema=response_schema_image
        )
    )
    return response.text

In [10]:
IMAGE_DIRECTORY = "/Users/sinngamkhaidem/Developer/Datasets/Multimodal-Sarcasm-Dataset/annotated/images"

In [11]:
curIndex = 0

In [20]:
for i in range(curIndex, df.shape[0]):
    if i%2 == 0:
        img = os.path.join(IMAGE_DIRECTORY, f"{df.iloc[i]['submissionId']}.png")
        response = detect_sarcasm_in_image(img)
        curIndex = i
        if len(response)>0:
            response_as_dict = json.loads(response)
            print(response_as_dict)
            df.at[i, 'i_s'] = response_as_dict[0]['isImageSarcastic?']

[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 1}]
[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 1}]
[{'isImageSarcastic?': 1}]
[{'isImageSarcastic?': 0}]
[{'isImageSarcastic?': 1}]
[{'isImageSarcastic?': 0}]


In [21]:
curIndex

198

In [25]:
new_df = df[df['i_s'].notnull()]
new_df.shape

(100, 6)

# Evaluation

In [24]:
def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy by comparing the true labels with the predicted labels.
    """
    correct_predictions = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    accuracy = correct_predictions / len(y_true)
    return accuracy

def compute_precision(y_true, y_pred):
    """
    Computes precision by calculating the ratio of true positives to the sum of true and false positives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    predicted_positives = sum(y_p == 1 for y_p in y_pred)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def compute_recall(y_true, y_pred):
    """
    Computes recall by calculating the ratio of true positives to the sum of true positives and false negatives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    actual_positives = sum(y_t == 1 for y_t in y_true)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def compute_f1(precision, recall):
    """
    Computes F1 score as the harmonic mean of precision and recall.
    """
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def evaluate(y_true, y_pred):
    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred)
    recall = compute_recall(y_true, y_pred)
    f1 = compute_f1(precision, recall)
    print("\tAccuracy: ", accuracy, "\n", "\tPrecision: ", precision,"\n", "\tRecall: ", recall, "\n", "\tF1 score: ", f1)
    


In [28]:
error_text = calculate_error_percentage(new_df["isImageSarcastic?"], new_df["i_s"])
error_text

20.0

In [27]:
new_df.to_csv("image_annotations.csv")